# Introduction
This notebook is a re-implementation of Fast AI's lesson 10 notebook.

The goal is to implement some basic matrix multiplication operations.

We are allowed to use the following libraries:
- Python and the included standard library
- matplotlib
- Jupyter notebooks and nbdev

In [1]:
from pathlib import Path
import pickle, gzip, math, os, time, shutil, matplotlib as mpl, matplotlib.pyplot as plt

# Get data
This section is copied from the existing notebook.

In [2]:
MNIST_URL = 'https://github.com/mnielsen/neural-networks-and-deep-learning/blob/master/data/mnist.pkl.gz?raw=true'
path_data = Path('data')
path_data.mkdir(exist_ok=True)
path_gz = path_data/'mnist.pkl.gz'

In [3]:
from urllib.request import urlretrieve
if not path_gz.exists(): urlretrieve(MNIST_URL, path_gz)

In [4]:
!ls -l data

total 33312
-rw-r--r--  1 pj  staff  17051982 May 24 19:17 mnist.pkl.gz


In [5]:
with gzip.open(path_gz, 'rb') as f: ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')

In [6]:
x_train[0].shape

(784,)

In [7]:
x_train[0][20:30]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32)

In [8]:
img0 = x_train[0]

# Task 1: Converting the 1D list to a 2D matrix
1. Using `yield`
2. Using `iter`/`islice`

## `yield`

In [9]:
def chunks(input_list, row_length):
  for i in range(0, len(input_list), row_length): yield input_list[i: i+row_length]

In [10]:
img0_yield_2d = list(chunks(img0, 28))

In [11]:
len(img0_yield_2d)

28

In [12]:
len(img0_yield_2d[0])

28

## `iter`/`islice`

In [13]:
from itertools import islice

In [29]:
iterator = iter(img0)
row_length = 28

img0_islice_2d = [list(islice(iterator, row_length)) for _ in range(len(img0) // row_length)]

In [30]:
len(img0_islice_2d)

28

In [31]:
len(img0_islice_2d[0])

28

## Compare `yield` and `islice`'s speed

In [32]:
%timeit -n 10 list(chunks(img0, 28))

4.66 µs ± 870 ns per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [33]:
%timeit -n 10 [list(islice(iterator, row_length)) for _ in range(len(img0) // row_length)]

5.06 µs ± 256 ns per loop (mean ± std. dev. of 7 runs, 10 loops each)


I find it interesting that `chunks` works slightly faster, but its standard deviation is much larger than that of `islice`.

# Implementing matrix

In [34]:
class Matrix:
    def __init__(self, matrix):
        self.matrix = matrix

    def __getitem__(self, idxs):
        return self.matrix[idxs[0]][idxs[1]]

In [35]:
img0_mat = Matrix(img0_islice_2d)

In [36]:
img0_mat[2,15]

0.0

In [37]:
print(img0_islice_2d[2][15])

0.0


Ok! Now we have implemented basic matrix operations in Python, we can use Pytorch.

In [38]:
import torch

tensor([[0.7777, 0.4367, 0.2872],
        [0.1293, 0.9236, 0.9745],
        [0.7217, 0.8311, 0.1779],
        [0.2846, 0.0288, 0.7082],
        [0.6324, 0.5387, 0.9834]])


/Users/pj/anaconda3/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
